In [58]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('./train.csv', sep=';')
test = pd.read_csv('./test.csv', sep=';')

answer = train.cardio

complete = pd.concat([train, test])
complete = complete.drop(['id', 'cardio'], axis=1)
complete.loc[complete['active'] == 'None', 'active'] = np.NaN
complete.loc[complete['alco'] == 'None', 'alco'] = np.NaN
complete.loc[complete['smoke'] == 'None', 'smoke'] = np.NaN
complete['active'] = complete['active'].astype('float')
complete['alco'] = complete['alco'].astype('float')
complete['smoke'] = complete['smoke'].astype('float')

vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,2))
vect_ap = pd.DataFrame(vectorizer.fit_transform(complete.ap_hi.astype('str') + complete.ap_lo.astype('str')).A)
complete = complete.drop(['ap_hi', 'ap_lo'], axis=1) 

complete = pd.concat([
    vect_ap,
    pd.DataFrame(complete.values)
], axis=1, ignore_index=True)
 
train = complete.head(len(train))
test = complete.tail(len(test))

from lightgbm.sklearn import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict, learning_curve

model = XGBClassifier(max_depth=5)
print model
predictions = cross_val_predict(model, train, answer, cv=5)
proba_predictions = cross_val_predict(model, train, answer, cv=5, method='predict_proba')

print log_loss(answer, proba_predictions)
print accuracy_score(answer, predictions)
print classification_report(answer, predictions)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
0.547294672541
0.731628571429
             precision    recall  f1-score   support

          0       0.72      0.77      0.74     35021
          1       0.75      0.69      0.72     34979

avg / total       0.73      0.73      0.73     70000



In [51]:
import time

model.fit(train, answer)
proba_predictions = model.predict_proba(test)

submission_response = pd.DataFrame({
    'y': proba_predictions[:,1]
})

timestr = time.strftime("%Y%m%d_%H%M")
pd.DataFrame(submission_response).to_csv('./outputs/typos.' + timestr + '.csv', header=False, index=False, line_terminator="\n")
pd.DataFrame(submission_response).to_csv('./outputs/_typos.latest.csv', header=False, index=False, line_terminator="\n")